In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Multiple_sclerosis"
cohort = "GSE141804"

# Input paths
in_trait_dir = "../../input/GEO/Multiple_sclerosis"
in_cohort_dir = "../../input/GEO/Multiple_sclerosis/GSE141804"

# Output paths
out_data_file = "../../output/preprocess/Multiple_sclerosis/GSE141804.csv"
out_gene_data_file = "../../output/preprocess/Multiple_sclerosis/gene_data/GSE141804.csv"
out_clinical_data_file = "../../output/preprocess/Multiple_sclerosis/clinical_data/GSE141804.csv"
json_path = "../../output/preprocess/Multiple_sclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Co-Morbid Multiple Sclerosis and Psoriasis: Clinical Outcomes and Gene Expression"
!Series_summary	"Psoriasis was found to ameliorate multiple sclerosis (MS) outcomes when it MS onset. However, the molecular basis for this observation remains unclear."
!Series_summary	"Herein, we compared the blood mononuclear cell transcriptome of psoriasis and MS comorbide (P/MS) patients with that of patients with either psoriasis or MS to understand the clinical observation."
!Series_overall_design	"A total of 45 patients (16 P/MS, 17 multiple sclerosis only (MSO), and 12 psoriasis only (PSO) patients) that met the inclusion criteria and 10 healthy control (HC) participants were analyzed."
Sample Characteristics Dictionary:
{0: ['gender: Male', 'gender: Female'], 1: ['age (years): 33.86', 'age (years): 19.76', 'age (years): 22.26', 'age (years): 35.35', 'age (years): 26.69', 'age (years): 52.16', 'age (years): 37.88', 'age (years): 46.78', 'age (years): 48.76'

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the dataset description, it contains gene expression data 
# from blood mononuclear cells, not just miRNA or methylation data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the background information, we have information about MS patients,
# but we don't see this information in the sample characteristics shown.
# Since we cannot identify where trait information is stored in the sample characteristics,
# we should consider it as not available in a usable form for this analysis.
is_trait_available = False
trait_row = None  # Trait data is not directly available in the sample characteristics shown

# For age, we can see age information in index 1
age_row = 1

# For gender, we can see gender information in index 0
gender_row = 0

# 2.2 Data Type Conversion
# For trait (if we had it), we would convert to binary (0 for control, 1 for MS)
def convert_trait(value):
    if value is None:
        return None
    # Extract the value part after the colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert based on expected values
    value = value.lower()
    if 'ms' in value or 'multiple sclerosis' in value or 'mso' in value or 'p/ms' in value:
        return 1
    elif 'control' in value or 'healthy' in value or 'hc' in value or ('pso' in value and 'p/ms' not in value):
        return 0
    else:
        return None

# For age, convert to continuous numeric value
def convert_age(value):
    if value is None:
        return None
    # Extract the value part after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to float, handling potential parsing errors
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

# For gender, convert to binary (0 for female, 1 for male)
def convert_gender(value):
    if value is None:
        return None
    # Extract the value part after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.lower()
    
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    else:
        return None

# 3. Save Metadata
# Initial filtering of dataset usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step since trait_row is None


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 3. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')

Gene data dimensions: 22277 genes × 46 samples


### Step 4: Gene Identifier Review

In [5]:
# Review gene identifiers
# The gene identifiers shown above (like '1007_s_at', '1053_at', etc.) are Affymetrix probe IDs, 
# not standard human gene symbols. These are probe identifiers from Affymetrix microarray platforms
# and need to be mapped to official gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the data from previous steps, I can observe that:
# - In gene_data, the index is probe IDs like '1007_s_at'
# - In gene_annotation, 'ID' column contains probe IDs and 'Gene Symbol' contains gene symbols

# 2. Extract the mapping between probe IDs and gene symbols
prob_col = 'ID'
gene_col = 'Gene Symbol'
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply the gene mapping to convert probe-level measurements to gene expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# 4. Print information about the mapped gene data
print(f"\nAfter mapping: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("\nFirst 5 gene symbols after mapping:")
print(gene_data.index[:5])



After mapping: 13830 genes × 46 samples

First 5 gene symbols after mapping:
Index(['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the index of gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 gene symbols after normalization: {normalized_gene_data.index[:5]}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Check if trait is available from previous steps
if is_trait_available:
    # Try to load clinical data if it exists
    clinical_data_file_exists = os.path.exists(out_clinical_data_file)
    
    if clinical_data_file_exists:
        clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
        print("Clinical data shape:", clinical_data.shape)
        
        # Fix clinical data format: ensure samples are rows and features are columns
        if trait in clinical_data.index:
            clinical_data = clinical_data.T
            print("Transposed clinical data to have samples as rows")
        
        # Link clinical and genetic data
        gene_data_t = normalized_gene_data.T
        linked_data = pd.concat([clinical_data, gene_data_t], axis=1, join='inner')
        print(f"Linked data shape before handling missing values: {linked_data.shape}")
        
        # Handle missing values
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
        
        # Determine if trait and demographic features are biased
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
    else:
        print("Clinical data file not found. Cannot link clinical and genetic data.")
        is_trait_available = False
        is_biased = True  # Set to True since we can't evaluate without trait data
else:
    print("Trait data was determined to be unavailable in previous steps.")
    is_biased = True  # Set to True since we can't evaluate without trait data
    
    # Create a minimal DataFrame with just gene expression data
    linked_data = normalized_gene_data.T
    print(f"Using only gene expression data, shape: {linked_data.shape}")

# Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from PBMC samples, but trait data for Multiple Sclerosis was not available in a usable format."
)

# Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associational studies due to missing trait data.")

Gene data shape after normalization: (13542, 46)
First 5 gene symbols after normalization: Index(['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS'], dtype='object', name='Gene')


Normalized gene data saved to ../../output/preprocess/Multiple_sclerosis/gene_data/GSE141804.csv
Trait data was determined to be unavailable in previous steps.
Using only gene expression data, shape: (46, 13542)
Dataset deemed not usable for associational studies due to missing trait data.
